In [1]:
%load_ext autoreload
%autoreload 2

import json
import logging
import os
import shutil
import subprocess
import time
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor, as_completed
from pathlib import Path

import libcst as cst
from tqdm import tqdm

from spot.data import GitRepo
from spot.type_env import collect_annots_info, mypy_checker
from spot.utils import proj_root, read_file, write_file

os.chdir(proj_root())

In [17]:
requests.get(
    f"https://api.github.com/search/repositories?q=language:python&sort=stars&order=desc&per_page=10"
).json()["items"][0]


{'id': 63476337,
 'node_id': 'MDEwOlJlcG9zaXRvcnk2MzQ3NjMzNw==',
 'name': 'Python',
 'full_name': 'TheAlgorithms/Python',
 'private': False,
 'owner': {'login': 'TheAlgorithms',
  'id': 20487725,
  'node_id': 'MDEyOk9yZ2FuaXphdGlvbjIwNDg3NzI1',
  'avatar_url': 'https://avatars.githubusercontent.com/u/20487725?v=4',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/TheAlgorithms',
  'html_url': 'https://github.com/TheAlgorithms',
  'followers_url': 'https://api.github.com/users/TheAlgorithms/followers',
  'following_url': 'https://api.github.com/users/TheAlgorithms/following{/other_user}',
  'gists_url': 'https://api.github.com/users/TheAlgorithms/gists{/gist_id}',
  'starred_url': 'https://api.github.com/users/TheAlgorithms/starred{/owner}{/repo}',
  'subscriptions_url': 'https://api.github.com/users/TheAlgorithms/subscriptions',
  'organizations_url': 'https://api.github.com/users/TheAlgorithms/orgs',
  'repos_url': 'https://api.github.com/users/TheAlgorithms/repos',
  'even

In [19]:
import dateparser

dateparser.parse("2022-04-20T00:00:01Z")


datetime.datetime(2022, 4, 20, 0, 0, 1, tzinfo=<StaticTzInfo 'Z'>)

In [31]:
import requests
import dateparser


def fetch_top_python_repos(n_pages: int, oldest_push: str):
    def request_page(page: int):
        return requests.get(
            f"https://api.github.com/search/repositories?q=language:python&pushed_at>{oldest_push}&sort=stars&order=desc&per_page=100&page={page}"
        ).json()

    pages = map(request_page, range(1, n_pages + 1))
    repos = list[GitRepo]()
    for page in tqdm(pages, desc="process pages"):
        if "items" not in page:
            print("Fetching page failed:")
            print(page)
            break
        for item in page["items"]:
            r = GitRepo(
                author=item["owner"]["login"],
                name=item["name"],
                url=item["html_url"],
                description=item["description"],
                stars=item["stargazers_count"],
                forks=item["forks_count"],
                last_update=dateparser.parse(item["pushed_at"]).replace(tzinfo=None),
            )
            repos.append(r)
    return repos


In [32]:
top_repos = fetch_top_python_repos(50, oldest_push="2021-04-20T00:00:01Z")
len(top_repos)


process pages: 10it [00:26,  2.61s/it]

Fetching page failed:
{'message': "API rate limit exceeded for 128.83.122.136. (But here's the good news: Authenticated requests get a higher rate limit. Check out the documentation for more details.)", 'documentation_url': 'https://docs.github.com/rest/overview/resources-in-the-rest-api#rate-limiting'}


1000

In [24]:
top_repos[0]


GitRepo(author='donnemartin', name='system-design-primer', url='https://github.com/donnemartin/system-design-primer', stars=182777, forks=33244, lines_of_code=None, last_update=datetime.datetime(2022, 5, 28, 11, 54, 39), n_type_annots=None, n_type_places=None)

In [3]:
all_repos = json.loads(read_file("data/mypy-dependents-by-stars.json"))
all_repos = [GitRepo.from_json(r) for r in all_repos]
# all_repos=all_repos[:10] # for testing


In [4]:
# download all candidate repos


def clear_downloaded_repos(repos_dir):
    shutil.rmtree(repos_dir)


def download_repos(
    to_download: list[GitRepo], repos_dir, download_timeout=10.0, max_workers=10
) -> list[GitRepo]:
    def download_single(repo: GitRepo):
        try:
            if repo.download(repos_dir, timeout=download_timeout):
                repo.read_last_update(repos_dir)
                return repo
            else:
                return None
        except subprocess.TimeoutExpired:
            return None
        except Exception as e:
            logging.warning(f"Failed to download {repo.name}. Exception: {e}")
            return None

    print("Downloading repos from Github...")
    t_start = time.time()
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        fs = [executor.submit(download_single, repo) for repo in to_download]
        rs = [f.result() for f in tqdm(as_completed(fs), total=len(fs))]
    print(f"Downloading took {time.time() - t_start} seconds.")
    downloaded = [r for r in rs if r is not None]
    return downloaded


datadir = Path(os.getenv("datadir"))
repos_dir = datadir / "SPOT-data/repos"
if not repos_dir.exists():
    (repos_dir / "downloading").mkdir(parents=True)
    (repos_dir / "downloaded").mkdir(parents=True)
    downloaded_repos = download_repos(all_repos, repos_dir)
    print("Deleting failed repos...")
    shutil.rmtree(repos_dir / "downloading")
else:
    print("Repos already downloaded.")
    downloaded_dirs = set(d.name for d in (repos_dir / "downloaded").iterdir())
    downloaded_repos = [r for r in all_repos if r.authorname() in downloaded_dirs]
    print("Reading last updates...")
    for r in tqdm(downloaded_repos):
        r.read_last_update(repos_dir)
print(f"Downloaded {len(downloaded_repos)}/{len(all_repos)} repos.")

# assert len(list((repos_dir / "downloaded").iterdir())) == len(downloaded_repos)


Repos already downloaded.
Reading last updates...


100%|██████████| 4890/4890 [00:27<00:00, 175.00it/s]

Downloaded 4890/5996 repos.


In [33]:
from datetime import datetime, timezone

date_threshold = datetime(2021, 4, 20)
new_repos = [r for r in downloaded_repos if r.last_update > date_threshold]
print(f"{len(new_repos)} / {len(downloaded_repos)} repos are updated within a year.")
loc_limit = 50000

small_repos = []
for rep in tqdm(new_repos):
    try:
        loc = rep.count_lines_of_code(repos_dir)
        if loc < loc_limit:
            small_repos.append(rep)
    except UnicodeDecodeError:
        # nothing we can do
        pass
    except Exception as e:
        logging.warning(f"Failed to count lines of code for {rep.name}. Exception: {e}")

print(
    f"{len(small_repos)}/{len(new_repos)} repos are within the size limit ({loc_limit} LOC)."
)


1218 / 4890 repos are updated within a year.


100%|██████████| 1218/1218 [00:05<00:00, 243.41it/s]

1181/1218 repos are within the size limit.


In [34]:
# filter out repos with too few annotations


def count_repo_annots(rep):
    try:
        rep.count_annotations(repos_dir)
        if rep.n_type_annots / rep.lines_of_code > 0.05:
            return rep
    except Exception as e:
        logging.warning(f"Failed to count annotations for {rep.name}. Exception: {e}")
        return None


with ProcessPoolExecutor(max_workers=30) as executor:
    fs = [executor.submit(count_repo_annots, rep) for rep in small_repos]
    rs = [f.result() for f in tqdm(as_completed(fs), total=len(fs))]
useful_repos: list[GitRepo] = [
    r for r in rs if r is not None and "typeshed" not in r.name
]

print(
    f"{len(useful_repos)}/{len(small_repos)} repos are parsable and have enough portions of type annotations."
)


  0%|          | 0/1181 [00:00<?, ?it/s]WARNING:root:Failed to count annotations for common-workflow-language. Exception: Syntax Error @ 24:23.
Incomplete input. Encountered 'linenum', but expected ';', or 'NEWLINE'.

                print linenum, l.rstrip()
                      ^
  1%|          | 6/1181 [00:00<03:12,  6.10it/s]WARNING:root:Failed to count annotations for rchain. Exception: Syntax Error @ 22:15.
Incomplete input. Encountered '"Test Plan:"', but expected ';', or 'NEWLINE'.

        print "Test Plan:", self.test_name
              ^
  1%|          | 14/1181 [00:02<03:14,  6.00it/s]WARNING:root:Failed to count annotations for Transcrypt. Exception: Syntax Error @ 1:1.
Incomplete input. Unexpectedly encountered '='.

= nonsense =
^
  2%|▏         | 26/1181 [00:03<01:55, 10.01it/s]WARNING:root:Failed to count annotations for canmatrix. Exception: Syntax Error @ 70:15.
Incomplete input. Encountered '"not yet working script for generating a communication layer for dedicated

664/1181 repos have enough portions of type annotations.


In [35]:
# Some summary statistics

# print total number of manual annotations
n_total_annots = sum(rep.n_type_annots for rep in useful_repos)
print("Total number of manual annotations:", n_total_annots)

# print total number of type places
n_total_places = sum(rep.n_type_places for rep in useful_repos)
print("Total number of type places:", n_total_places)

# print total number of lines of code
n_total_lines = sum(rep.lines_of_code for rep in useful_repos)
print("Total number of lines of code:", n_total_lines)

# print average number of type annotations per line of code excluding projects with more than 1000 lines of code
n_avg_annots = (
    sum(rep.n_type_annots for rep in useful_repos if rep.lines_of_code < 1000)
    / n_total_lines
)


Total number of manual annotations: 343595
Total number of type places: 544497
Total number of lines of code: 3342911


[GitRepo(author='skorokithakis', name='catt', url='https://github.com/skorokithakis/catt', stars=1740, forks=762, lines_of_code=2036, last_update=datetime.datetime(2022, 4, 10, 1, 30, 43), n_type_annots=140, n_type_places=433),
 GitRepo(author='encode', name='databases', url='https://github.com/encode/databases', stars=769, forks=48, lines_of_code=3124, last_update=datetime.datetime(2022, 3, 6, 12, 25, 10), n_type_annots=323, n_type_places=498),
 GitRepo(author='Curt-Park', name='rainbow-is-all-you-need', url='https://github.com/Curt-Park/rainbow-is-all-you-need', stars=490, forks=110, lines_of_code=107, last_update=datetime.datetime(2022, 1, 13, 23, 4, 48), n_type_annots=26, n_type_places=30),
 GitRepo(author='jreese', name='aiomultiprocess', url='https://github.com/jreese/aiomultiprocess', stars=585, forks=45, lines_of_code=1140, last_update=datetime.datetime(2022, 2, 4, 21, 28, 7), n_type_annots=138, n_type_places=213),
 GitRepo(author='instaloader', name='instaloader', url='https:/

In [5]:
import pickle

useful_repos_path = proj_root() / "scripts" / "useful_repos.pkl"
with useful_repos_path.open("wb") as f:
    pickle.dump(useful_repos, f)
print(f"Saved {len(useful_repos)} useful repos to {useful_repos_path}.")
with useful_repos_path.open("rb") as f:
    print(pickle.load(f)[:3])


[GitRepo(author='typeddjango', name='pytest-mypy-plugins', url='https://github.com/typeddjango/pytest-mypy-plugins', stars=12, forks=0, lines_of_code=1039, last_update=datetime.datetime(2022, 4, 18, 23, 25, 40), n_type_annots=155, n_type_places=158), GitRepo(author='jfly', name='jfly.github.io', url='https://github.com/jfly/jfly.github.io', stars=0, forks=0, lines_of_code=650, last_update=datetime.datetime(2022, 4, 12, 8, 23, 39), n_type_annots=39, n_type_places=122), GitRepo(author='seattleflu', name='id3c', url='https://github.com/seattleflu/id3c', stars=2, forks=0, lines_of_code=8883, last_update=datetime.datetime(2022, 4, 21, 15, 38, 59), n_type_annots=675, n_type_places=1068)]


In [1]:
# Run form here: process GithubRepos into SrcDatasets.

%load_ext autoreload
%autoreload 2

import pickle

from spot import proj_root

repos_split_path = proj_root() /  "data/repos_split.pkl"
with repos_split_path.open("rb") as f:
    repos_split = pickle.load(f)

In [10]:
import os

from spot.data import SrcDataset, get_dataset_name, PreprocessArgs
import spot.function_dataset as fd
from spot.utils import Path, run_long_task, DefaultTokenizer, not_none
import subprocess

datadir = Path(not_none(os.getenv("datadir")))
repos_dir = datadir / "SPOT-data/repos/"

quicktest = False

func_only = True
pre_args = PreprocessArgs(
    drop_comments=True,
    imports_in_preamble=True,
    stub_in_preamble=False,
)
test_tag = "quicktest-" if quicktest else ""
src_datasets_path = (
    datadir / "SPOT-data" / (test_tag + get_dataset_name(pre_args, func_only))
)
src_datasets_path.mkdir(parents=True, exist_ok=True)

src_datasets: dict[str, SrcDataset] = {}
with run_long_task("Generating SrcDatasets"):
    for name, repos in repos_split.items():
        if quicktest:
            repos = repos[:3]
        repo_paths = [r.repo_dir(repos_dir) for r in repos]
        if func_only:
            src_data = fd.dataset_from_repos(
                repos_dir / "downloaded", repo_paths, pre_args
            )
        else:
            src_data = SrcDataset.from_repos(
                repos_dir / "downloaded",
                repo_paths,
                pre_args,
            )
        for s in src_data.all_srcs:
            assert len(s.types) > 0, f"{s.file} has no labels."
        src_datasets[name] = src_data
        with open(src_datasets_path / f"{name}.pkl", "wb") as f:
            pickle.dump(src_data, f)
    print("Saved source datasets to:", src_datasets_path)
    subprocess.run(["du", "-sh", src_datasets_path])


Starting task: Generating SrcDatasets


Generating dataset from repos: 100%|██████████| 50/50 [00:29<00:00,  1.69it/s]


Saved source datasets to: /mnt/data0/jiayi/SPOT-data/func_datasets-v0-PreprocessArgs()
555M	/mnt/data0/jiayi/SPOT-data/func_datasets-v0-PreprocessArgs()
Pushover: (Finished: 'Generating SrcDatasets'.) Time taken: 509.1s


In [11]:
src_datasets["train"].print_stats()

num_repos: 567
num_files: 109437
num_lines: 3278314
num_labels: 264710
main_tokens_per_file:
   mean: 1344.7
   median: 453
   min: 19
   max: 100968
preamble_tokens_per_file:
   mean: 116.82
   median: 85
   min: 1
   max: 1225
target_tks_per_file:
   mean: 8.715
   median: 6
   min: 2
   max: 725


In [18]:
print(str(src_datasets["train"].all_srcs[1001]))

========TokenizedSrc========
file:zabuldon__teslajsonpy/teslajsonpy/homeassistant/gps/Odometer.__init__
repo:zabuldon__teslajsonpy
--------Preamble--------
from typing import Optional, Text
from teslajsonpy.homeassistant.vehicle import VehicleDevice

--------Main Code--------
# teslajsonpy.homeassistant.vehicle/VehicleDevice
def _name(self):
    return (
        f"{self._display_name} {self.type}"
        if self._display_name is not None and self._display_name!= self._vin[-6:]
        else f"Tesla Model {str(self._vin[3]).upper()} {self.type}"
    )
# teslajsonpy.homeassistant.vehicle/VehicleDevice
def _uniq_name(self):
    return f"Tesla Model {str(self._vin[3]).upper()} {self._vin[-6:]} {self.type}"
# teslajsonpy.homeassistant.power/EnergySiteDevice
def _name(self):
    return f"{self._site_name} {self.type}"
# teslajsonpy.homeassistant.power/EnergySiteDevice
def _uniq_name(self):
    return self._name()
# teslajsonpy.homeassistant.gps/Odometer
def __init__(self, data, controller):


In [12]:
max(src_datasets["train"].all_srcs, key=lambda s: len(s.tokenized_code)).print_code(500)

# fastapi.routing/APIRouter
def api_route(
    self,
    path,
    *,
    response_model = None,
    status_code = None,
    tags = None,
    dependencies = None,
    summary = None,
    description = None,
    response_description = "Successful Response",
    responses = None,
    deprecated = None,
    methods = None,
    operation_id = None,
    response_model_include = None,
    response_model_exclude = None,
    response_model_by_alias = True,
    response_model_exclude_unset = False,
    response_model_exclude_defaults = False,
    response_model_exclude_none = False,
    include_in_schema = True,
    response_class = Default(JSONResponse),
    name = None,
    callbacks = None,
    openapi_extra = None,
    generate_unique_id_function = Default(
        generate_unique_id
    ),
):
    def decorator(func):
        self.add_api_route(
            path,
            func,
            response_model=response_model,
            status_code=status_code,
            tags=tags,
        

In [4]:
src_datasets["train"].print_stats()


num_repos: 572
num_files: 16281
num_lines: 2698669
num_labels: 295457
main_tokens_per_file:
   mean: 1311.6
   median: 608
   min: 8
   max: 55437
preamble_tokens_per_file:
   mean: 301.97
   median: 181
   min: 6
   max: 9784
target_tks_per_file:
   mean: 65.318
   median: 28
   min: 2
   max: 3798
n_files_too_wide: 444
too_wide_ratio: 0.014897
preprocess: PreprocessArgs(imports_in_preamble=True, stub_in_preamble=True, drop_comments=True)


In [8]:
def preamble_len(src):
    return len(src.preamble_code.split("\n"))


weird_src = max(src_datasets["train"].all_srcs, key=preamble_len)


In [11]:
import plotly.express as px
from pandas import DataFrame

from spot.utils import cumulative_counts

len_counts = [len(src.tokenized_code) for src in src_datasets["train"].all_srcs]
xs, ys = cumulative_counts(len_counts)
px.line(
    DataFrame({"tokens_per_file": xs, "n_files": ys}), x="tokens_per_file", y="n_files"
)
